In [1]:
import geopandas as gpd

## Get County Data

In [2]:
file_path = "data/cb_2018_us_county_500k/cb_2018_us_county_500k.shp"
counties_gdf = gpd.read_file(file_path)

In [3]:
counties_gdf = counties_gdf.to_crs(9311)

## Open McDonalds Data

In [4]:
mcdonalds_gdf = gpd.read_file("data/mcdonalds_va_h3_5.gpkg")
mcdonalds_gdf = mcdonalds_gdf.to_crs(9311)

## Get Dollar General Data

In [5]:
kfcs_gdf = gpd.read_file("data/kfcs.gpkg")
kfcs_gdf = kfcs_gdf.to_crs(9311)

## Intersect Data

In [6]:
counties_mcdonalds_count_gdf = counties_gdf.join(
    gpd.sjoin(mcdonalds_gdf, counties_gdf)
    .groupby("index_right")
    .size()
    .rename("stores"),
    how="left",
)

In [7]:
counties_mcdonalds_count_gdf = counties_mcdonalds_count_gdf.fillna(0)
counties_mcdonalds_count_gdf["mcdonalds"] = counties_mcdonalds_count_gdf[
    "stores"
].astype(int)
counties_mcdonalds_count_gdf = counties_mcdonalds_count_gdf[
    ["GEOID", "geometry", "mcdonalds", "NAME"]
]

In [8]:
counties_kfcs_count_gdf = counties_gdf.join(
    gpd.sjoin(kfcs_gdf, counties_gdf).groupby("index_right").size().rename("stores"),
    how="left",
)

In [12]:
counties_kfcs_count_gdf = counties_kfcs_count_gdf.fillna(0)
counties_kfcs_count_gdf["kfcs"] = counties_kfcs_count_gdf["stores"].astype(int)
counties_kfcs_count_gdf = counties_kfcs_count_gdf[["GEOID", "geometry", "kfcs", "NAME"]]

#### Merge Data now

In [13]:
mcd_kfc_gdf = counties_mcdonalds_count_gdf.merge(
    counties_kfcs_count_gdf[["GEOID", "kfcs"]],
    on="GEOID",
    how="left",
)

In [14]:
def mcdonalds_dollar_general_classification(row) -> str:
    kfc = row["kfcs"]
    md = row["mcdonalds"]
    if kfc == 0 and md == 0:
        return "no stores"
    if kfc == md:
        return "same amount"
    if md > kfc:
        return "more mcdonald's"
    if md < kfc:
        return "more dollar generals"

In [18]:
mcd_kfc_gdf["md_or_kfc"] = mcd_kfc_gdf.apply(
    lambda row: mcdonalds_dollar_general_classification(row), axis=1
)

In [19]:
mcd_kfc_gdf = mcd_kfc_gdf.to_crs(9311)

In [20]:
mcd_kfc_gdf.to_file("data/mcd_kfc_gdf.gpkg")